In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
799,800,3453965,9.174260e+05,8/7/2006 8:00:00 AM,2110,FINCH AVE W,KISKADEE DR,NaN,Major Arterial,Etobicoke York,...,NaN,NaN,NaN,1,West Humber-Clairville,1,West Humber-Clairville (1),D23,612888.2303,4.843422e+06
10605,10606,80206765,4.001880e+09,4/12/2014 8:00:00 AM,2030,GREENWOOD AVE,OAKVALE AVE,NaN,Minor Arterial,Toronto and East York,...,NaN,NaN,NaN,65,Greenwood-Coxwell,65,Greenwood-Coxwell (65),D55,634479.4869,4.837646e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INVAGE', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['MANOEUVER', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 18)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INJURY,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158
13983,9/30/2017 8:00:00 AM,1311,ROGERS RD,MCROBERTS AVE,Etobicoke York,43.683287,-79.455447,Clear,Daylight,Dry,Non-Fatal Injury,Rear End,NaN,NaN,NaN,NaN,109,Caledonia-Fairbank
12717,7/6/2016 8:00:00 AM,1831,401 C W MCCOWAN N RAMP RD,MCCOWAN RD,Scarborough,43.783583,-79.255756,Clear,Daylight,Dry,Fatal,SMV Other,NaN,NaN,"Automobile, Station Wagon",Driving Properly,128,Agincourt South-Malvern West


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,...,INJURY,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH
14075,2017-10-27 08:00:00,2031,FINCH AVE W,KIPLING AVE,Etobicoke York,43.743193,-79.582073,Clear,Dark,Dry,...,Major,NaN,"Automobile, Station Wagon",Driving Properly,3,Thistletown-Beaumond Heights,Friday,20,2017,10
14647,2018-06-19 08:00:00,1700,MADAWASKA AVE,WILLOWDALE AVE,North York,43.795069,-79.412708,Clear,Daylight,Dry,...,Minor,NaN,Pick Up Truck,Driving Properly,50,Newtonbrook East,Tuesday,17,2018,6
11716,2015-07-15 08:00:00,0429,1500 EGLINTON AVE W,NaN,North York,43.697718,-79.440814,Clear,Dark,Dry,...,NaN,NaN,"Automobile, Station Wagon",Other,107,Oakwood Village,Wednesday,4,2015,7


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Twilight',
    'Dawn, artificial': 'Twilight',
    'Dawn': 'Twilight',
    'Dusk': 'Twilight',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    10936
Dark         7298
Twilight      719
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'SnowIce',
    'Packed Snow': 'SnowIce',
    'Slush': 'SnowIce',
    'Ice': 'SnowIce',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry        15389
Wet         3141
SnowIce      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,...,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH,SEVERE_COLLISION
11349,2015-01-30,DON VALLEY PARKWAY S,LAWRENCE AVE E,North York,43.740245,-79.332248,Clear,Dark,Dry,Non-Fatal Injury,...,0.0,NaN,NaN,42,Banbury-Don Mills,Friday,19,2015,1,0
2367,2007-08-10,278 GREENWOOD AVE,IVY Aven,Toronto and East York,43.673245,-79.328990,Clear,Dark,Dry,Non-Fatal Injury,...,0.0,Other,NaN,70,South Riverdale,Friday,2,2007,8,1
8801,2012-10-10,SHEPPARD AVE E,WARDEN AVE,Scarborough,43.778745,-79.307890,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,Truck - Open,Failed to Yield Right of Way,118,Tam O'Shanter-Sullivan,Wednesday,14,2012,10,0
16545,2020-07-11,SPADINA AVE,FORT YORK BLVD,Toronto and East York,43.639853,-79.392726,Clear,Daylight,Wet,Non-Fatal Injury,...,0.0,"Automobile, Station Wagon",Driving Properly,165,Harbourfront-CityPlace,Saturday,18,2020,7,0
15940,2019-09-12,BURNHAMTHORPE RD,THE WEST MALL,Etobicoke York,43.644480,-79.567203,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,"Automobile, Station Wagon",Driving Properly,13,Etobicoke West Mall,Thursday,10,2019,9,0


In [34]:
def categorize_hour(hour):
    if 0 <= hour < 8:
        return 'LateNight'
    elif 8 <= hour < 14:
        return 'MorningAfternoon'
    elif 14 <= hour < 19:
        return 'Evening'
    else:
        return 'Night'

In [35]:
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [36]:
def categorize_vehicle(v):
    if v in ["Automobile, Station Wagon", "Passenger Van", "Taxi", "Other"]:
        return "Car"
    elif v in [
        "Pick Up Truck", "Truck – Open", "Truck – Closed (Blazer, etc)",
        "Truck – Dump", "Truck–Tractor", "Delivery Van",
        "Tow Truck", "Truck (other)", "Truck – Car Carrier",
        "Construction Equipment"
    ]:
        return "Truck"
    elif v in [
        "Municipal Transit Bus (TTC)", "Bus (Other) (Go Bus, Gray Coa)",
        "Intercity Bus", "School Bus"
    ]:
        return "Bus"
    elif v in ["Motorcycle", "Bicycle", "Moped", "Rickshaw"]:
        return "Two Wheeler"
    elif v in ["Police Vehicle", "Fire Vehicle", "Ambulance", "Other Emergency Vehicle"]:
        return "Emergency"
    else:
        return "Car"

In [37]:
def categorize_drivact(a):
    if a == "DrivingProperly":
        return "Driver Proper"
    elif a in ["Failed to Yield Right of Way", "Disobeyed Traffic Control", "Wrong Way on One Way Road"]:
        return "Right-of-Way/Traffic Violation"
    elif a in ["Exceeding Speed Limit", "Speed too Fast For Condition", "Speed too Slow"]:
        return "Speed-Related"
    elif a in ["Lost control", "Improper Turn", "Following too Close", "Improper Lane Change", "Improper Passing"]:
        return "Control/Maneuver Error"
    else:
        return "Other/Unknown"


In [38]:
def categorize_impact(impact):
    if impact in ["Pedestrian Collisions", "Cyclist Collisions"]:
        return "Pedestrian/Cyclist"
    elif impact == "Rear End":
        return "Rear End"
    elif impact in ["Sideswipe", "Approaching"]:
        return "Sideswipe/Approaching"
    elif impact in ["Angle", "Turning Movement"]:
        return "T-Bone"
    elif impact in ["SMV Other", "SMV Unattended Vehicle"]:
        return "Slow/Parked"
    else:
        return "Other/Unknown"

In [39]:
df['TIME_OF_DAY'] = df['HOUR'].apply(categorize_hour)
df['SEASON'] = df['MONTH'].apply(categorize_season)
df["VEHICLE_TYPE"] = df["VEHTYPE"].apply(categorize_vehicle)
df["DRIVER_ACTION"] = df["DRIVACT"].apply(categorize_drivact)
df["IMPACT_TYPE"] = df["IMPACTYPE"].apply(categorize_impact)

In [40]:
df = df[df["IMPACT_TYPE"] != "Other/Unknown"]

In [41]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 1), 'ROAD_CONDITION'] = 'Wet'
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 0), 'ROAD_CONDITION'] = 'Dry'

In [42]:
df['LIGHT'] = df['LIGHT'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 1), 'LIGHT'] = 'Dark'
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 0), 'LIGHT'] = 'Daylight'

In [43]:
df['VISIBILITY'] = df['VISIBILITY'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 1), 'VISIBILITY'] = 'Rain'
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 0), 'VISIBILITY'] = 'Clear'

In [44]:
df.shape

(18726, 27)

In [45]:
df.to_csv('ksi_collisions.csv', index=False)